In [ ]:
# !python3 -m pip install matplotlib

In [ ]:
# !python3 -m pip install pandas

In [ ]:
# !python3 -m pip install mysql-connector

In [ ]:
# !python3 -m pip install pymysql

## GET API

In [ ]:
# key = 'GWXLIMM8PJ1W2F2C'

In [1]:
## FUNCTION
import pymysql

api_url = 'https://www.alphavantage.co/query'
    
def get_alphavantage(params_dict: dict()) -> dict():
    params = '&'.join([f'{key}={value}' for key, value in params_dict.items()])
    print(params)
    
    import requests
    
    url = f'{api_url}?{params}'
    r = requests.get(url)
    data = r.json()
    return data

def init_db() -> pymysql.connections.Connection:
    # Open database connection
    # Connect to the database
    db = pymysql.connect(
        host='alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com',
         user='admin',
         password='adminadmin',
         database='alphavantage_uat',
         cursorclass=pymysql.cursors.DictCursor
    )
    return db

def insert_data(df, table_name, db):
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    
    field_len = len(df.columns.values)
    cols = ','.join(list(df.columns.values))

    for i, row in df.iterrows():
        print(i)
        data_row = ','.join([row[list(df.columns.values)[field_i]] for field_i in range(field_len)])

        sql = f"""
            INSERT IGNORE INTO {table_name} ({cols})
            VALUES ({data_row});
        """
        print(sql.format(table_name,cols,data_row))

        cursor.execute(sql)
        
db = init_db()

In [ ]:
## daily ticker/currency (/api/dailyeod)
params_dict = {
    'function': 'FX_DAILY',
    'from_symbol': 'EUR',
    'to_symbol': 'USD',
    'apikey': 'demo'
}
data = get_alphavantage(params_dict)
# data

# df = pd.read_json(fname).reset_index()
df = pd.DataFrame(data).reset_index()

df = pd.concat([
    df[df.columns.tolist()[-1]].dropna().apply(pd.Series),
    df.drop(df.columns.tolist()[-1], axis=1)
], axis=1)

df = df[
    df[df.columns.tolist()[-1]].isnull() # take out last column with null value (Meta Data) - logic will change if json structure changes
].drop(
    df.columns.tolist()[-1], axis=1 # drop empty column (Meta Data)
)
df.rename(columns={'index': 'date'}, inplace=True)
df.columns = [x.replace('.','_').replace(' ','').lower() for x in df.columns]
df['date'] = df['date'].apply(lambda x: f"'{x}'")

df = df.reset_index(drop=True)
# df

table_name = '_'.join(list(params_dict.values())[:-1]).lower()
print(table_name)

insert_data(df, table_name, db)
db.commit()

In [3]:
## monthly CPI (/api/monthlycpi)
params_dict = {
    'function': 'CPI',
    'interval': 'monthly',
    'apikey': 'demo'
}
data = get_alphavantage(params_dict)
# data

# df = pd.read_json(fname)
df = pd.DataFrame(data)

df = pd.concat([
    df[df.columns.tolist()[-1]].apply(pd.Series),
    df.drop(df.columns.tolist()[-1], axis=1)
], axis=1)
df = df[['date','value']]
df['date'] = df['date'].apply(lambda x: f"'{x}'")

df = df.reset_index(drop=True)
# df

table_name = '_'.join(list(params_dict.values())[:-1]).lower()
print(table_name)

insert_data(df, table_name, db)
db.commit()

function=CPI&interval=monthly&apikey=demo
cpi_monthly
0

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-09-01',274.310);
        
1

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-08-01',273.567);
        
2

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-07-01',273.003);
        
3

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-06-01',271.696);
        
4

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-05-01',269.195);
        
5

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-04-01',267.054);
        
6

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-03-01',264.877);
        
7

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2021-02-01',263.014);
        
8

            INSERT IGNORE INTO cpi_monthly (date,value)

111

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2012-06-01',229.478);
        
112

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2012-05-01',229.815);
        
113

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2012-04-01',230.085);
        
114

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2012-03-01',229.392);
        
115

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2012-02-01',227.663);
        
116

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2012-01-01',226.665);
        
117

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2011-12-01',225.672);
        
118

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2011-11-01',226.230);
        
119

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2011-10-01',22

220

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2003-05-01',183.5);
        
221

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2003-04-01',183.8);
        
222

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2003-03-01',184.2);
        
223

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2003-02-01',183.1);
        
224

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2003-01-01',181.7);
        
225

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2002-12-01',180.9);
        
226

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2002-11-01',181.3);
        
227

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2002-10-01',181.3);
        
228

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('2002-09-01',181.0);
        
2


            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1994-03-01',147.2);
        
331

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1994-02-01',146.7);
        
332

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1994-01-01',146.2);
        
333

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1993-12-01',145.8);
        
334

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1993-11-01',145.8);
        
335

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1993-10-01',145.7);
        
336

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1993-09-01',145.1);
        
337

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1993-08-01',144.8);
        
338

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1993-07-01',144.4);
        
339



437

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1985-04-01',106.9);
        
438

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1985-03-01',106.4);
        
439

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1985-02-01',106.0);
        
440

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1985-01-01',105.5);
        
441

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1984-12-01',105.3);
        
442

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1984-11-01',105.3);
        
443

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1984-10-01',105.3);
        
444

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1984-09-01',105.0);
        
445

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1984-08-01',104.5);
        
4

...

764

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1958-01-01',28.6);
        
765

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-12-01',28.4);
        
766

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-11-01',28.4);
        
767

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-10-01',28.3);
        
768

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-09-01',28.3);
        
769

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-08-01',28.3);
        
770

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-07-01',28.3);
        
771

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-06-01',28.1);
        
772

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1957-05-01',28.0);
        
773

     

873

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-12-01',24.1);
        
874

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-11-01',24.2);
        
875

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-10-01',24.4);
        
876

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-09-01',24.5);
        
877

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-08-01',24.5);
        
878

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-07-01',24.4);
        
879

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-06-01',24.1);
        
880

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-05-01',23.9);
        
881

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1948-04-01',23.8);
        
882

     

980

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1940-01-01',13.9);
        
981

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-12-01',14);
        
982

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-11-01',14);
        
983

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-10-01',14);
        
984

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-09-01',14.1);
        
985

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-08-01',13.8);
        
986

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-07-01',13.8);
        
987

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-06-01',13.8);
        
988

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1939-05-01',13.8);
        
989

           

1089

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-12-01',16.1);
        
1090

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-11-01',16.4);
        
1091

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-10-01',16.5);
        
1092

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-09-01',16.6);
        
1093

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-08-01',16.5);
        
1094

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-07-01',16.6);
        
1095

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-06-01',16.8);
        
1096

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-05-01',16.9);
        
1097

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1930-04-01',17);
        
109

1196

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1922-01-01',16.9);
        
1197

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-12-01',17.3);
        
1198

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-11-01',17.4);
        
1199

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-10-01',17.5);
        
1200

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-09-01',17.5);
        
1201

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-08-01',17.7);
        
1202

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-07-01',17.7);
        
1203

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-06-01',17.6);
        
1204

            INSERT IGNORE INTO cpi_monthly (date,value)
            VALUES ('1921-05-01',17.7);
        
1

## CLEAN DATA

## QUERY

In [12]:
import pandas as pd

query = f"SELECT * FROM alphavantage_uat.fx_daily_eur_usd;"
df = pd.read_sql(query, db)
df

,date,1_open,2_high,3_low,4_close


In [2]:
import pandas as pd

query = f"SELECT * FROM alphavantage_uat.cpi_monthly;"
df = pd.read_sql(query, db)
df

,date,value
0,1913-01-01,9.800
1,1913-02-01,9.800
2,1913-03-01,9.800
3,1913-04-01,9.800
4,1913-05-01,9.700
...,...,...
1300,2021-05-01,269.195
1301,2021-06-01,271.696
1302,2021-07-01,273.003
1303,2021-08-01,273.567


## CREATE TABLE - Adhoc

In [ ]:
# cursor = db.cursor()
    
# # Drop table if it already exist using execute() method.
# cursor.execute(f"DROP TABLE IF EXISTS fx_daily_eur_usd;")

# # Create table as per requirement
# sql = f"""
#     CREATE TABLE IF NOT EXISTS fx_daily_eur_usd (
#         date DATE PRIMARY KEY,
#         1_open FLOAT,
#         2_high FLOAT,
#         3_low FLOAT,
#         4_close FLOAT
#     );
# """

# cursor.execute(sql)

In [ ]:
# cursor = db.cursor()

# # Drop table if it already exist using execute() method.
# cursor.execute(f"DROP TABLE IF EXISTS cpi_monthly;")

# # Create table as per requirement
# sql = f"""
#     CREATE TABLE IF NOT EXISTS cpi_monthly (
#         date DATE PRIMARY KEY,
#         value FLOAT
#     );
# """

# cursor.execute(sql)

In [2]:
cursor = db.cursor()

cursor.execute("SHOW DATABASES;")

for i in cursor:
    print(i)

{'Database': 'alphavantage'}
{'Database': 'alphavantage_uat'}
{'Database': 'information_schema'}
{'Database': 'mysql'}
{'Database': 'performance_schema'}
{'Database': 'sys'}


In [3]:
cursor = db.cursor()

cursor.execute("SHOW TABLES;")

for tables in cursor:
    print(tables)

{'Tables_in_alphavantage_uat': 'cpi_monthly'}
{'Tables_in_alphavantage_uat': 'fx_daily_eur_usd'}
{'Tables_in_alphavantage_uat': 'table_name'}


In [6]:
cursor = db.cursor()

cursor.execute(f"DESCRIBE cpi_monthly;")

for field in cursor:
    print(field)

{'Field': 'date', 'Type': 'date', 'Null': 'NO', 'Key': 'PRI', 'Default': None, 'Extra': ''}
{'Field': 'value', 'Type': 'float', 'Null': 'YES', 'Key': '', 'Default': None, 'Extra': ''}


## MONTHLY

In [ ]:
# import requests

# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={key}'
# r = requests.get(url)
# data = r.json()

# print(data)

## CONNECT TO MySQL RDS

In [ ]:
# import pymysql

In [ ]:
# # Connect to the database
# connection = pymysql.connect(host='alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com',
#                              user='admin',
#                              password='adminadmin')
# #                              database='alphavantage',
# #                              cursorclass=pymysql.cursors.DictCursor)
# connection

In [ ]:
# cursor = connection.cursor()
# cursor

In [ ]:
# # execute SQL query using execute() method.
# cursor.execute("SELECT VERSION();")

# # Fetch a single row using fetchone() method.
# ver = cursor.fetchone()
# print ("Database version : %s " % ver)

## CREATE DATABASE

In [ ]:
# connection.cursor().execute('CREATE DATABASE alphavantage_uat;')

## SHOW DATABASES

In [ ]:
# cursor.execute("SHOW DATABASES;")

# for databases in cursor:
#     print(databases[0])

## THIRD PARTY API

In [ ]:
# from alpha_vantage.timeseries import TimeSeries
# import matplotlib.pyplot as plt

# ts = TimeSeries(key='GWXLIMM8PJ1W2F2C', output_format='json')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# print(data)
# print(meta_data)
# # data['close'].plot()
# # plt.title('Intraday Times Series for the MSFT stock (1 min)')
# # plt.show()